In [1]:
import requests
from dotenv import load_dotenv
import os 
from pprint import pprint
import json
import pandas as pd
from tmdbv3api import Movie, TMDb as tmdb
import time

#load hidden credentials and values
load_dotenv() 
imdbapi_key = os.getenv("tmdbapikey")
#master_list = pd.read_csv("1990_2020_movie_list1227.csv", encoding='latin1')
master_list = pd.read_csv("tmdb_ids.csv", encoding='latin1')

print(len(master_list.index))
master_list.head(5)
#imdbapi_key

4740


,tmdb_id,imdb_id,title,year,Unnamed: 4
0,12244,tt0472033,9,2009,NaN
1,8065,tt0478087,21,2008,NaN
2,109410,tt0453562,42,2013,NaN
3,3682,tt0120577,54,1998,NaN
4,1271,tt0416449,300,2007,NaN


In [4]:
#imdb_id_list = master_list.movieid.tolist()
imdb_id_list = master_list.imdb_id.tolist()
titles = master_list.title.tolist()
imdb_id_list[:10]

['tt0472033',
 'tt0478087',
 'tt0453562',
 'tt0120577',
 'tt0416449',
 'tt0450385',
 'tt8579674',
 'tt1190080',
 'tt1022603',
 'tt0147800']

In [6]:
runlist = []

for i, imdbid in enumerate(imdb_id_list):
    time.sleep(0.1)
    try:
#         imdbid = "tt11100856"
        #rqst = f"https://api.themoviedb.org/3/find/{imdbid}?api_key={imdbapi_key}&external_source=imdb_id"
#         rqst = f"https://api.themoviedb.org/3/movie/{imdbid}?api_key={imdbapi_key}&external_source=imdb_id"
#         r = requests.get(rqst)
#         mid = r.json()['movie_results'][0]['id']
#         rqst2 = f"https://api.themoviedb.org/3/movie/{mid}?api_key={imdbapi_key}&language=en-US"
#GET /movie/{movie_id}/credits
        rqst2 = f"https://api.themoviedb.org/3/movie/{imdbid}/credits?api_key={imdbapi_key}&language=en-US"
#         print(rqst2)
        runlist.append(requests.get(rqst2).json())
        print(titles[i])

    except:
        print(f"no luck with {titles[i]}")
# df_norm = pd.json_normalize(runlist)
# df_norm.iloc[1,:]

9
21
42
54
300
1408
1917
2012
(500) Days of Summer
10 Things I Hate About You
10,000 BC
101 Dalmatians
102 Dalmatians
12 Rounds
12 Strong
12 Years a Slave
127 Hours
13 Going on 30
1492: Conquest of Paradise
15 Minutes
16 Blocks
17 Again
2 Days in Paris
2 Days in the Valley
2 Fast 2 Furious
2 Guns
200 Cigarettes
2015 Oscar Nominated Short Films: Animation
21 & Over
21 Bridges
21 Grams
21 Jump Street
22 Jump Street
24 Hour Party People
25th Hour
27 Dresses
28 Days
28 Days Later
28 Weeks Later
29th Street
3 Days to Kill
3 Men and a Little Lady
3 Ninjas
3 Ninjas Kick Back
3 Strikes
3:10 to Yuma
30 Days of Night
30 Minutes or Less
300: Rise of an Empire
3000 Miles to Graceland
40 Days and 40 Nights
47 Meters Down: Uncaged
47 Ronin
5 Flights Up
50 First Dates
50 to 1
50/50
7 Days in Entebbe
8 Heads in a Duffel Bag
8 Mile
8 Seconds
8 Women
88 Minutes
8MM
A Beautiful Day in the Neighborhood
A Beautiful Mind
A Bronx Tale
A Bug's Life
A Christmas Carol
A Cinderella Story
A Civil Action
A Cry in 

In [7]:
#save json to file
with open('castcrew.json', 'w') as outfile:
    json.dump(runlist, outfile)

In [4]:
#retrieve json
with open('castcrew.json', 'r') as outfile:
    runlist = json.load(outfile)

TypeError: the JSON object must be str, bytes or bytearray, not TextIOWrapper

In [9]:
outls = []
for rl in runlist:
    castdf = pd.json_normalize(rl['cast'])
    castdf['credtype'] = 'cast'
    crewdf = pd.json_normalize(rl['crew'])
    crewdf['credtype'] = 'crew'
    tmpdf = pd.concat([castdf, crewdf], ignore_index=True, sort=False)
    tmpdf['tmdb_id'] = rl['id']
    outls.append(tmpdf)

In [10]:
outdf = pd.concat(outls, ignore_index=True, sort=False)

# castdf = pd.json_normalize(runlist[0]['cast']).head()
# castdf['id']
outdf.head()

,adult,gender,id,known_for_department,name,original_name,popularity,profile_path,cast_id,character,credit_id,order,credtype,department,job,tmdb_id
0,False,2.0,109.0,Acting,Elijah Wood,Elijah Wood,8.178,/7UKRbJBNG7mxBl2QQc5XsAh6F8B.jpg,4.0,9 (voice),52fe44d29251416c750422cf,0.0,cast,NaN,NaN,12244
1,False,2.0,4764.0,Acting,John C. Reilly,John C. Reilly,4.536,/sShFIjzRt9DKiJG6lyUEKPkocQk.jpg,5.0,5 (voice),52fe44d29251416c750422d3,1.0,cast,NaN,NaN,12244
2,False,1.0,6161.0,Acting,Jennifer Connelly,Jennifer Connelly,12.046,/egh1eOHuYgeoqdlLQgaXMl6cPOm.jpg,6.0,7 (voice),52fe44d29251416c750422d7,2.0,cast,NaN,NaN,12244
3,False,2.0,290.0,Acting,Christopher Plummer,Christopher Plummer,2.893,/iZh3s9Vy9vYD4DYnAda6C1kdeco.jpg,7.0,1 (voice),52fe44d29251416c750422db,3.0,cast,NaN,NaN,12244
4,False,2.0,1064.0,Acting,Crispin Glover,Crispin Glover,3.577,/imBnLpSXvg61qDDdEfvL6R4ITKt.jpg,8.0,6 (voice),52fe44d29251416c750422df,4.0,cast,NaN,NaN,12244


In [11]:
outdf.to_csv("cast_crew.csv")

In [28]:
giveme = ['Director', 'Epk Director', 'Epk Producer', 'Executive Music Producer', 'Executive Producer', \
          'Music', 'Music Director', 'Music Editor', 'Music Producer', \
          'Original Music Composer', 'Producer', 'Songs', 'Writer']
castlists = outdf[outdf['credtype']=='cast'].groupby(['tmdb_id'])['name'].apply(', '.join).reset_index()
castlists.columns = ["tmdb_id", "cast"]
crewlist  = outdf[["tmdb_id", "name", "job"]][outdf['job'].isin(giveme)].drop_duplicates()
crewlist = crewlist.pivot_table(index="tmdb_id", 
                          columns="job", values="name",
                          aggfunc=lambda x: ', '.join(x)).reset_index()
castcrew = pd.merge(castlists, crewlist, how="outer",on="tmdb_id")
castcrew.head()

,tmdb_id,cast,Director,Epk Director,Epk Producer,Executive Music Producer,Executive Producer,Music,Music Director,Music Editor,Music Producer,Original Music Composer,Producer,Songs,Writer
0,5,"Tim Roth, Jennifer Beals, Antonio Banderas, Va...","Allison Anders, Alexandre Rockwell, Robert Rod...",NaN,NaN,NaN,"Alexandre Rockwell, Quentin Tarantino",NaN,NaN,Denise Okimoto,Mark Mothersbaugh,Combustible Edison,Lawrence Bender,NaN,"Allison Anders, Alexandre Rockwell, Robert Rod..."
1,6,"Emilio Estevez, Cuba Gooding Jr., Denis Leary,...",Stephen Hopkins,NaN,NaN,NaN,"Marilyn Vance, Lloyd Segan",NaN,NaN,NaN,NaN,Alan Silvestri,Gene Levy,NaN,NaN
2,12,"Albert Brooks, Ellen DeGeneres, Alexander Goul...",Andrew Stanton,NaN,NaN,NaN,John Lasseter,NaN,NaN,Bill Bernstein,NaN,Thomas Newman,Graham Walters,NaN,NaN
3,13,"Tom Hanks, Robin Wright, Gary Sinise, Mykelti ...",Robert Zemeckis,NaN,NaN,Joel Sill,NaN,NaN,NaN,Kenneth Karman,NaN,Alan Silvestri,"Wendy Finerman, Steve Tisch, Steve Starkey",NaN,NaN
4,14,"Kevin Spacey, Annette Bening, Thora Birch, Wes...",Sam Mendes,NaN,NaN,NaN,NaN,NaN,NaN,"Bill Bernstein, Joanie Diener",NaN,Thomas Newman,"Bruce Cohen, Dan Jinks",NaN,NaN


In [29]:
printout = pd.merge(master_list, castcrew, how="outer",on="tmdb_id") 
printout.to_csv("castcrew.csv")

In [ ]:
df_norm['production_companies_number'] = df_norm.production_companies.apply(lambda x: len(x))
df_norm['production_countries_number'] = df_norm.production_countries.apply(lambda x: len(x))
df_norm['spoken_languages_number'] = df_norm.spoken_languages.apply(lambda x: len(x))
df_norm['genres'] = df_norm.genres.apply(lambda x: "|".join([y['name'] for y in x]))
df_norm['production_companies'] = df_norm.production_companies.apply(lambda x: "|".join([y['name'] for y in x]))
df_norm['production_countries'] = df_norm.production_countries.apply(lambda x: "|".join([y['name'] for y in x]))
df_norm['spoken_languages'] = df_norm.spoken_languages.apply(lambda x: "|".join([y['english_name'] for y in x]))

In [ ]:
colorder = ['imdb_id', 'id', 'title', 'original_title', 'budget', 'revenue', 
            'genres', 'imdb_id', 'original_language', 'overview', 
            'production_companies', 'production_countries', 'release_date', 
            'runtime', 'spoken_languages', 'status', 'tagline', 'popularity', 
            'vote_average', 'vote_count', 'production_companies_number', 
            'production_countries_number', 'spoken_languages_number']

df_norm[colorder].to_csv("scraped_results.csv", index=False)

In [ ]:
# baseurl = "https://imdb-api.com/API/"
# #trailerext = "Trailer/"
# #top250ext = "Top250Movies/"

# filmid = "tt2488496"

# trailer_URL = f"{baseurl}{trailerext}{imdbapi_key}/{filmid}"

# response = requests.request("GET", trailer_URL)

# trailer_df = pd.json_normalize(json.loads(response.text))
# trailer_df

In [43]:
from youtube_search import YoutubeSearch

#imdb_id_list = master_list.tmdb_id.astype(int).tolist()
#titles
runlist = []

for i,t in enumerate(titles):
#t = titles[0]
    try:
        imdbid = imdb_id_list[i]
        txt = f"official movie trailer {t}"
        #print(txt)
        results = YoutubeSearch(txt, max_results=5).to_dict()
        tmpdf = pd.json_normalize(results)
        tmpdf['imdb_id'] = imdbid
        tmpdf['imdb_title'] = t
        runlist.append(tmpdf)
    except:
        pass
    
runlist[-1]
#results

,id,thumbnails,title,long_desc,channel,duration,views,url_suffix,imdb_id,imdb_title
0,MorOaD61KUI,[https://i.ytimg.com/vi/MorOaD61KUI/hq2.jpg?sq...,Orlando (1992) Theatrical Trailer,Director: Sally Potter,spinteractive1,1:16,"284,039 views",/watch?v=MorOaD61KUI,557734,Orlando
1,gUmTvO9KHHU,[https://i.ytimg.com/vi/gUmTvO9KHHU/hq2.jpg?sq...,Orlando Trailer 1993,Orlando Trailer,Video Detective,1:27,"178,666 views",/watch?v=gUmTvO9KHHU,557734,Orlando
2,moNH4N44D28,[https://i.ytimg.com/vi/moNH4N44D28/hq2.jpg?sq...,Kingdom Of Heaven - Official® Trailer [HD],"Release Date: May 6, 2005 ""Kingdom of Heaven"" ...",TrailersPlaygroundHD,2:42,"1,074,685 views",/watch?v=moNH4N44D28,557734,Orlando
3,E9OgBzdzEsY,[https://i.ytimg.com/vi/E9OgBzdzEsY/hq2.jpg?sq...,Digging For Fire Official Trailer #1 (2015) - ...,Subscribe to,Movieclips Trailers,2:16,"1,007,991 views",/watch?v=E9OgBzdzEsY,557734,Orlando
4,T0P4eW-DLiI,[https://i.ytimg.com/vi/T0P4eW-DLiI/hq2.jpg?sq...,ZULU - Official Trailer,ZULU CAST,Pathe,1:42,"158,194 views",/watch?v=T0P4eW-DLiI,557734,Orlando


In [44]:
yt_trailer_df = pd.concat(runlist, ignore_index=True, sort=False)
yt_trailer_df.head()

,id,thumbnails,title,long_desc,channel,duration,views,url_suffix,imdb_id,imdb_title
0,_qApXdc1WPY,[https://i.ytimg.com/vi/_qApXdc1WPY/hq2.jpg?sq...,9 - Official Trailer,Coming to theatres 9.9.09.,Focus Features,2:31,"3,868,625 views",/watch?v=_qApXdc1WPY,12244,9
1,UY7Tge7ftxM,[https://i.ytimg.com/vi/UY7Tge7ftxM/hq2.jpg?sq...,9 - Teaser Trailer,Coming to theatres 9.9.09.,Focus Features,2:07,"348,719 views",/watch?v=UY7Tge7ftxM,12244,9
2,_qyw6LC5pnE,[https://i.ytimg.com/vi/_qyw6LC5pnE/hq2.jpg?sq...,FAST AND FURIOUS 9 Trailer (2020),First,FilmSelect Trailer,3:51,"55,463,130 views",/watch?v=_qyw6LC5pnE,12244,9
3,8Qn_spdM5Zg,[https://i.ytimg.com/vi/8Qn_spdM5Zg/hq2.jpg?sq...,Star Wars: The Rise of Skywalker | Final Trailer,"The saga will end, the story lives forever. Wa...",Star Wars,2:38,"45,738,208 views",/watch?v=8Qn_spdM5Zg,12244,9
4,ccaNMcPqpQ0,[https://i.ytimg.com/vi/ccaNMcPqpQ0/hq2.jpg?sq...,WRONG TURN Official Trailer (NEW 2021) Horror ...,WRONG TURN,ONE Media,2:35,"2,099,159 views",/watch?v=ccaNMcPqpQ0,12244,9


In [53]:
import datetime

def convertdur(x):
    if x:
        cntcl = str(x).count(":")
        if cntcl == 1:
            r = datetime.datetime.strptime(x, "%M:%S")
        elif cntcl == 2:
            r = datetime.datetime.strptime(x, "%H:%M:%S")
        a_timedelta = r - datetime.datetime(1900, 1, 1)
        return a_timedelta.total_seconds()
    

yt_trailer_df['seconds'] = yt_trailer_df['duration'].apply(convertdur)

In [54]:
yt_trailer_df.head()

,id,thumbnails,title,long_desc,channel,duration,views,url_suffix,imdb_id,imdb_title,seconds
0,_qApXdc1WPY,[https://i.ytimg.com/vi/_qApXdc1WPY/hq2.jpg?sq...,9 - Official Trailer,Coming to theatres 9.9.09.,Focus Features,2:31,"3,868,625 views",/watch?v=_qApXdc1WPY,12244,9,151.0
1,UY7Tge7ftxM,[https://i.ytimg.com/vi/UY7Tge7ftxM/hq2.jpg?sq...,9 - Teaser Trailer,Coming to theatres 9.9.09.,Focus Features,2:07,"348,719 views",/watch?v=UY7Tge7ftxM,12244,9,127.0
2,_qyw6LC5pnE,[https://i.ytimg.com/vi/_qyw6LC5pnE/hq2.jpg?sq...,FAST AND FURIOUS 9 Trailer (2020),First,FilmSelect Trailer,3:51,"55,463,130 views",/watch?v=_qyw6LC5pnE,12244,9,231.0
3,8Qn_spdM5Zg,[https://i.ytimg.com/vi/8Qn_spdM5Zg/hq2.jpg?sq...,Star Wars: The Rise of Skywalker | Final Trailer,"The saga will end, the story lives forever. Wa...",Star Wars,2:38,"45,738,208 views",/watch?v=8Qn_spdM5Zg,12244,9,158.0
4,ccaNMcPqpQ0,[https://i.ytimg.com/vi/ccaNMcPqpQ0/hq2.jpg?sq...,WRONG TURN Official Trailer (NEW 2021) Horror ...,WRONG TURN,ONE Media,2:35,"2,099,159 views",/watch?v=ccaNMcPqpQ0,12244,9,155.0


In [55]:
yt_trailer_df.to_csv("youtube_trailer_deets.csv")

In [ ]:
# import os
# import google.oauth2.credentials as gcred
# import google.auth.transport.requests
# from google_auth_oauthlib.flow import InstalledAppFlow 
# import googleapiclient.discovery
# import datetime
# import json

# def get_goog_oathcreds(token_txt_dir, creds_json_dir):
#     if not os.path.exists(token_txt_dir):
#         flow = InstalledAppFlow.from_client_secrets_file(creds_json_dir,
#                     scopes=["https://www.googleapis.com/auth/youtube.force-ssl"], 
#                             redirect_uri=r"http://localhost/")

#         oathcreds = flow.run_local_server()

#         with open(token_txt_dir, 'w+') as f:
#             json.dump({"refresh_token": oathcreds._refresh_token, "expiry": oathcreds.expiry, \
#                         "token": oathcreds.token, "id_token": oathcreds.id_token}, f, indent=4, sort_keys=True, default=str)
#     else:
#         temp = json.loads(open(token_txt_dir, 'r').read())
#         mysecret = json.loads(open(creds_json_dir, 'r').read())['installed']
#         oathcreds = gcred.Credentials(
#             temp['token'],
#             refresh_token=temp['refresh_token'],
#             id_token=temp['id_token'],
#             token_uri=mysecret['token_uri'],
#             client_id=mysecret['client_id'],
#             client_secret=mysecret['client_secret'],
#             scopes=["https://www.googleapis.com/auth/youtube.force-ssl"], #########################################
#         )
#         tmpexpiry = temp['expiry'].split('.')[0]
#         oathcreds.expiry = datetime.datetime.strptime(tmpexpiry,'%Y-%m-%d %H:%M:%S')
#         request = google.auth.transport.requests.Request()
#         if oathcreds.expired:
#             oathcreds.refresh(request)
#     return oathcreds

# def create_yt_instance(oathcreds):
#     api_service_name = "youtube"
#     api_version = "v3"
#     youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=oathcreds)
#     return youtube

# #https://python.gotrained.com/youtube-api-extracting-comments/
# def get_video_comments(service, **kwargs):
#     comments = []
#     results = service.commentThreads().list(**kwargs).execute()

#     while results:
#         for item in results['items']:
#             comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
#             comments.append(comment)

#         if 'nextPageToken' in results:
#             kwargs['pageToken'] = results['nextPageToken']
#             results = service.commentThreads().list(**kwargs).execute()
#         else:
#             break

#     return comments


# oathcreds = get_goog_oathcreds("youtubetoken.json","youtube.json")
# youtube = create_yt_instance(oathcreds)

# vidid = results[0]["url_suffix"].split("=")[-1]
# vidid

In [ ]:
# comments = get_video_comments(youtube, part='snippet', videoId=vidid, textFormat='plainText')
# pprint(comments)

In [ ]:
accum = []

In [11]:
for ttid in imdb_id_list:
# ttid = "tt0035423"
    try:
        thisurl = f"https://www.imdb.com/title/{ttid}/soundtrack"
        page = requests.get(thisurl).text
        soup = BeautifulSoup(page, "html.parser")
        stdiv = soup.find('div' , id="soundtracks_content").div
        for c in stdiv.find_all("div"):
            strs = list(c.stripped_strings)
            if strs:
                accum.append([ttid] + strs)
    except:
        pass

In [13]:
accum[:10]

[]

In [ ]:
from __future__ import unicode_literals
import youtube_dl

yturls = pd.read_csv(r"yt_urls.csv", encoding="latin1")

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}
ydl = youtube_dl.YoutubeDL(ydl_opts)

In [ ]:
urlist = yturls.values.tolist()

for urlend, ttid in urlist:
    try:
        ydl.download(['http://www.youtube.com/watch?v=sGbxmsDFVnE'])
    except:
        pass